# Enunciado
Hola ingeniero 👋. Bienvenido a tu tercer desafio! Ya sabes que en [Vault-Tec Corporation](https://fallout.fandom.com/es/wiki/Vault-Tec_Corporation) tenemos sensores encargados de monitorear las condiciones actuales de nuestros refugios. En ocasiones pasadas detectamos que varios de ellos presentaban fallas recurrentes, impidiendo que pudieramos determinar las condiciones de nuestros bóvedas y darle la seguridad esperada a nuestros clientes en un mundo post apocaliptico 🙃. 

Anteriormente, nuestro equipo de mantenimiento fisico pudo reemplazar los sensores mal funcionantes e hicimos un codigo para poder determinar cuanto fue el tiempo de falla registrado por un sensor para una fecha, de la siguiente forma:

```python
get_sensor_failure_duration("hidra_flow", "2025-07-31", logs)
> {'sensor_id': 'hidra_flow', 'date': datetime.date(2025, 7, 31), 'failure_total_time': -94}
```

Ahora lo que requiere Vault-Tec Corporation es que construyas un [pipeline de datos](https://www.ibm.com/mx-es/think/topics/data-pipeline) el cual automatize lo desarrollado en ocasiones pasadas: 

extraccion de logs -> determinar tiempo de falla (en segundos) para un sensor -> determinar probabilidad de falla del sensor -> determinar probabilidad de falla condicional de dos sensores.  

Tu tarea, si decides aceptarla, es diseñar y construir dicho pipeline.

# Insumos
El equipo encargado del monitoreo de la calidad de los sensores, te ha enviado los [logs](https://keepcoding.io/blog/que-son-logs-y-para-que-sirven/) de dichos sensores. 

Algunos ejemplos de logs de estos sensores se ven de la siguiente forma:
```python
[
    {'sensor_id': 'oxy_guard','event_type': 'FAILURE_START','timestamp': '2025-07-31 07:01:24','duration_seconds': 577},
    {'sensor_id': 'radi_shield','event_type': 'FAILURE_START','timestamp': '2025-07-31 07:19:02','duration_seconds': 542}   
]
```

Recuerda que la vez pasada el equipo de monitoreo se encargo de agrupar los logs, para que solo exista una entrada de inicio y fin de falla por sensor por dia.

# Tu turno!
Ahora es tu turno! Vault-Tec Corp necesita saber lo siguiente:

1. Un reporte semanal:
- Dado un sensor y una fecha de inicio, determina cuántas horas ha permanecido en estado de falla **cada dia de la semana** comprendida entre la fecha de inicio y los 7 días siguientes.
    - En caso de que uno o mas de los dias no existan dentro de los logs asignales un valor de `None` al resultado.
    - Ejemplo: El sensor radi_shield ha fallado un total de 89H en la última semana.

2. Riesgo semanal de falla:
- Dado un sensor y una fecha de inicio, determina la probabilidad de que el sensor falle para **cada dia** de la semana analizada.
    - En caso de que uno o mas de los dias no existan dentro de los logs asignales un valor de `None` al resultado.


3. Riesgo semanal condicional de falla:
- Dado dos sensores, un numero de horas y una fecha, determina la probabilidad de que un sensor falle dado que otro sensor también falló por mas de las horas definidas.
    - En caso de que el dia solicitado no este presente en los logs asignale un valor de `None` al resultado.

💡 Recuerda que todos estos puntos ya los hemos desarollado previamente, ahora el reto es pensar en como integrar y conectarlo todo.

Para poder lograrlo tu equipo te recomienda seguir los siguientes pasos:

1. Crear una clase para la entidad Log: Crea una clase para poder almacenar la informacion correspondiente a un log.

💡 Revisa nuevamente que atributos componen a un log. 

In [1]:
from datetime import datetime
from typing import Literal

from pydantic import BaseModel


class Log(BaseModel):
    sensor_id: Literal["oxy_guard", "thermo_core", "hidra_flow", "radi_shield"]
    event_type: Literal["FAILURE_END", "FAILURE_START"]
    timestamp: datetime
    duration_seconds: int

2. Extraer los logs: Crea una clase cuya responsabilidad sea extraer y filtrar informacion de un archivo de logs.

💡 Recuerda nombrar tus clases con nombres significativos, tal y como lo dice uno de los principios del core de python: "explicito es mejor que implicito".

In [2]:
import json
from datetime import date
from os.path import abspath


class LogExtractor:
    def __init__(self, file_path: str):
        self.file_abspath = abspath(file_path)

    def from_file(self) -> list[Log]:
        with open(self.file_abspath) as file:
            raw_logs: list[dict] = json.load(file)
        logs: list[Log] = [Log(**log) for log in raw_logs]
        return logs

    @staticmethod
    def fetch_logs_on_date(date: date, logs: list[Log], sensor_id: int | None = None) -> list[dict]:
        logs_on_date: list[Log] = list(
            filter(
                lambda x: (
                    (x.timestamp.date() == date) and (sensor_id is None or x.sensor_id == sensor_id)
                ),
                logs,
            )
        )
        return logs_on_date

3. Analizar las fallas por medio de los logs: Crea una clase que tenga los metodos necesarios para poder analizar las fallas presentadas por los sensores segun lo requerido por Vault-Tec Corporation.

In [ ]:
from datetime import date, timedelta


class SensorFailureAnalyzer:
    @staticmethod
    def get_sensor_failure_duration(
        sensor_id: str, searched_date: date, logs: list[Log]
    ) -> float | None:
        logs_on_date: list[Log] = LogExtractor.fetch_logs_on_date(
            sensor_id=sensor_id, date=searched_date, logs=logs
        )

        if not logs_on_date:
            return None

        failure_start_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_START"), 0
        )
        failure_end_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_END"), 0
        )

        return failure_end_time - failure_start_time

    def get_weekly_sensor_failure_duration(
        sensor_id: int, start_date: date, logs: list[Log], end_date: date | None = None
    ) -> list[float]:
        end_date = start_date + timedelta(7) if end_date is None else end_date
        searched_week: list[date] = [
            (start_date + timedelta(days=i)) for i in range((end_date - start_date).days)
        ]
        weekly_sensor_failure_duration = [
            {
                "sensor_id": sensor_id,
                "date": str(date),
                "failure_seconds": SensorFailureAnalyzer.get_sensor_failure_duration(
                    sensor_id=sensor_id, searched_date=date, logs=logs
                ),
            }
            for date in searched_week
        ]
        return weekly_sensor_failure_duration

    def get_conditional_failure_probability(
        sensor_a_id: int,
        sensor_b_id: int,
        searched_date: date,
        failure_hours_threshold: int | None = 3,
    ) -> float: ...

In [4]:
from datetime import datetime

searched_date = datetime.strptime("2025-08-01", "%Y-%m-%d").date()

extraction = LogExtractor(
    r"/Users/spuertaf/Desktop/Repos/HenryLectures/LoopsYFunciones/homework/data/logs.json"
)
SensorFailureAnalyzer.get_weekly_sensor_failure_duration(
    sensor_id="thermo_core",
    start_date=datetime.strptime("2025-07-31", "%Y-%m-%d").date(),
    logs=extraction.from_file(),
)

[252, -133, 518, -536, -204, -12, 19]